In [1]:
import os
import PyPDF2

def get_attachments(reader):
    """
    Retrieves the file attachments of the PDF as a dictionary of file names
    and the file data as a bytestring.
    :return: dictionary of filenames and bytestrings
    """
    catalog = reader.trailer["/Root"]
    embedded_files = catalog['/Names']['/EmbeddedFiles']['/Names']
    attachments = {}
    for i in range(0, len(embedded_files), 2):
        name = embedded_files[i]
        file_spec = embedded_files[i + 1].get_object()
        file_data = file_spec['/EF']['/F'].get_data()
        if not name.lower().endswith('.pdf'):
            name += '.pdf'
        attachments[name] = file_data

    return attachments

def save_attachments(pdf_path, save_directory=None):
    # Extract the base name of the PDF file without the extension
    base_name = os.path.splitext(os.path.basename(pdf_path))[0]

    # Use the provided save directory or the current working directory
    if save_directory:
        base_name = os.path.join(save_directory, base_name)
    else:
        base_name = os.path.join(os.getcwd(), base_name)
    
    # Create a directory with the base name of the PDF file
    try:
        if not os.path.exists(base_name):
            os.makedirs(base_name)
    except PermissionError:
        print(f"Permission denied: Cannot create directory '{base_name}'.")
        return

    # Open the PDF file
    with open(pdf_path, 'rb') as handler:
        reader = PyPDF2.PdfReader(handler)
        attachments = get_attachments(reader)

        # Save the attachments to files in the new directory
        for f_name, f_data in attachments.items():
            file_path = os.path.join(base_name, f_name)
            try:
                with open(file_path, 'wb') as outfile:
                    outfile.write(f_data)
            except PermissionError:
                print(f"Permission denied: Cannot write file '{file_path}'.")

# Path to the PDF file
pdf_path = 'Form PAS-3-15112021_signed.pdf'

# Optional: Specify a different save directory
# save_directory = 'C:\\Users\\YourUsername\\Documents'

# Save attachments to the directory
save_attachments(pdf_path)
